##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - Common use cases
This notebook demonstrates the basic task that Gemma can solve by using the right prompting.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/%5BGemma_1%5DCommon_use_cases.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Gemma setup

To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run
  the Gemma 2B model.
* Generate and configure a Kaggle username and an API key as Colab secrets.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.


### Configure your credentials

Add your your Kaggle credentials to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create new secrets: `KAGGLE_USERNAME` and `KAGGLE_KEY`
3. Copy/paste your username into `KAGGLE_USERNAME`
3. Copy/paste your key into `KAGGLE_KEY`
4. Toggle the buttons on the left to allow notebook access to the secrets.


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

### Install dependencies
Run the cell below to install all the required dependencies.

In [ ]:
!pip install -q -U keras keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible

## Exploring prompting capabilities

### Gemma

**About Gemma**

Gemma is a family of lightweight, state-of-the-art open models from Google, built from the same research and technology used to create the Gemini models. They are text-to-text, decoder-only large language models, available in English, with open weights, pre-trained variants, and instruction-tuned variants. Gemma models are well-suited for a variety of text generation tasks, including question answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as a laptop, desktop or your own cloud infrastructure, democratizing access to state of the art AI models and helping foster innovation for everyone.

**Prompt formatting**

Instruction-tuned (IT) models are trained with a specific formatter that annotates all instruction tuning examples with extra information, both at training and inference time. The formatter has two purposes:

* Indicating roles in a conversation, such as the system, user, or assistant roles.
* Delineating turns in a conversation, especially in a multi-turn conversation.

Below, we specify the control tokens used by Gemma and their use cases. Note that the control tokens are reserved in and specific to our tokenizer.

* Token to indicate a user turn: `user`
* Token to indicate a model turn: `model`
* Token to indicate the beginning of dialogue turn: `<start_of_turn>`
* Token to indicate the end of dialogue turn: `<end_of_turn>`

Here's the [official documentation](https://ai.google.dev/gemma/docs/formatting) regarding prompting instruction-tuned models.

In [ ]:
import keras
import keras_nlp
from IPython.display import display, Markdown, Latex

os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get("KAGGLE_KEY")

In [ ]:
# Let's load Gemma using Keras
gemma_model_id = "gemma_1.1_instruct_2b_en"
gemma = keras_nlp.models.GemmaCausalLM.from_preset(gemma_model_id)

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_1.1_instruct_2b_en/3' to your Colab notebook...
Attaching 'metada

#### Prompting example: Summarization

In [ ]:
prompt = """<start_of_turn>user

Text:
Tech giant Google, originally a Stanford research project by Larry Page and Sergey Brin,
has become synonymous with searching the web. Their complex algorithms prioritize relevant
results, making Google the go-to platform for information seekers. Beyond search,
Google offers email (Gmail), document creation tools (Google Docs), mapping (Google Maps),
the Android operating system for smartphones, and the Chrome web browser.
A leader in AI, Google ventures into self-driving cars and quantum computing.
Their influence is so vast, "to google" has become a verb for searching online.

Could you summarize the text in a few words?<end_of_turn>
<start_of_turn>model"""

response = gemma.generate(prompt, max_length=512)
display(Markdown(response[len(prompt) :]))

Summary:
Google is a tech giant known for its search engine, email, documents, maps, and more, encompassing various products and services.

#### Prompting example: Reasoning

In [ ]:
prompt = """<start_of_turn>user
Think and write your step-by-step reasoning before responding.

Explain why the sky is blue.<end_of_turn>
<start_of_turn>model"""

response = gemma.generate(prompt, max_length=1024)
display(Markdown(response[len(prompt) :]))

**Step 1: Understand the physics of light scattering.**

- Light is composed of different wavelengths, each with a distinct color.
- When sunlight enters the atmosphere, it is scattered by molecules and particles in the air.


**Step 2: Explain the blue wavelength's scattering effect.**

- Blue light has a shorter wavelength than other colors in the spectrum.
- When blue light interacts with molecules in the atmosphere, it is scattered more strongly than other colors.


**Step 3: Describe the scattering process.**

- Blue light is scattered more efficiently by nitrogen molecules than by oxygen molecules.
- This means that more blue light reaches our eyes from the sky than other colors.


**Step 4: Quantify the effect.**

- The blue sky effect is most pronounced on a clear, sunny day with a high concentration of nitrogen molecules in the atmosphere.
- Studies have shown that the blue sky effect accounts for about 47% of the visible light that reaches our eyes from the sky.


**Step 5: Conclusion.**

- The blue sky is caused by the scattering of blue light by molecules in the atmosphere.
- This scattering effect is responsible for the blue color we perceive in the sky.

#### Prompting example: Classification

In [ ]:
prompt = """<start_of_turn>user
Classify the text into neutral, negative, or positive.
Generate only the class, nothing else.
Text: Text: I think the food was awesome.<end_of_turn>
<start_of_turn>model
Sentiment:"""

response = gemma.generate(prompt, max_length=64)
print("Sentiment: " + response[len(prompt) :])

Sentiment:  Positive


#### Prompting example: Extraction

In [ ]:
text = """Lost in a worn Berlin travel guide, Ania underlined Warsaw and London.
Two cities, two dreams: a bustling career in finance and
a cozy bookshop tucked away in a cobbled London street."""

prompt = f"""<start_of_turn>user
Your task is to find all the city names in the given text.
Your response is an array of the model names in the format [\"city_name\"].
If you don't find model names in the abstract or you are not sure, return [\"\"]

Text:
{text}<end_of_turn>
<start_of_turn>model"""

response = gemma.generate(prompt, max_length=512)
print("Extracted cities: " + response[len(prompt) :])

Extracted cities: ["Berlin", "Warsaw", "London"]


#### Prompting example: Multi-turn dialogue

In [ ]:
import textwrap


def format_messages(messages):
    fmt_messages = [
        f"<start_of_turn>{author}\n{text}<end_of_turn>" for author, text in messages
    ]
    fmt_messages.append("<start_of_turn>model")
    return "\n".join(fmt_messages)


history = [
    (
        "user",
        "You're a helpful assistant. Answer user's question in a friendly and helpful way. "
        "Please introduce yourself first.",
    )
]

print("\n*** YOU'RE ENTERING CHAT MODE. TYPE 'EXIT' TO FINISH THE CHAT ***\n")
while True:
    prompt = format_messages(history)
    response = gemma.generate(prompt, max_length=512)
    response = response[len(prompt) :]

    history.append(("model", response))
    print(textwrap.fill(f"Gemma: {response} \n", 120))

    question = input("User: ")
    if question == "EXIT":
        break

    print()
    history.append(("user", question))


*** YOU'RE ENTERING CHAT MODE. TYPE 'EXIT' TO FINISH THE CHAT ***

Gemma:  Hi there! I'm your friendly AI assistant here to help you with any questions or tasks you may have. I'm here
to assist you with information, provide solutions, and make your experience more efficient. Let's get started!
User: Who are you?

Gemma:  I am a large language model, trained by Google. I am designed to provide helpful and informative responses
based on the vast amount of data I have been trained on. I am here to assist you with your questions and requests to the
best of my abilities.
User: EXIT


#### Prompting example: Code Generation
_Note: While Gemma offers some basic code generation capabilities, a more specialized model called CodeGemma is introduced in the next section for these tasks._

In [ ]:
prompt = """<start_of_turn>user
Generate a Python function that multiplies two numbers<end_of_turn>
<start_of_turn>model"""

response = gemma.generate(prompt, max_length=1024)
display(Markdown(response[len(prompt) :]))

```python
def multiply(num1, num2):
  """
  This function multiplies two numbers.

  Args:
    num1: The first number.
    num2: The second number.

  Returns:
    The product of num1 and num2.
  """

  product = num1 * num2
  return product
```

**Example Usage:**

```python
# Multiply 5 and 3
product = multiply(5, 3)

# Print the product
print(product)  # Output: 15
```

**Explanation:**

* The function takes two arguments, `num1` and `num2`.
* Inside the function, we use the multiplication operator (`*`) to calculate the product of `num1` and `num2`.
* The product is stored in the `product` variable.
* The function returns the calculated product.

**Additional Notes:**

* This function only works for positive numbers. For negative numbers, a `ZeroDivisionError` will be raised.
* The function is designed to be simple and efficient. It does not handle errors or special cases.

In [ ]:
# We need to restart the session to release GPU memory
# before loading different model
get_ipython().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}